In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

# plots in LaTeX-style
import tueplots
from tueplots import bundles
plt.rcParams.update(bundles.neurips2022())

# Data loading

We start by loading the data as in **data_exploration.ipynb**

In [ ]:
# load data
data = pd.read_csv("../data/Food_Production.csv", sep=";")

In [17]:
data.head()

,Food product,Plant based,Land Use Change,Animal Feed,Farm,Processing,Transport,Packaging,Retail,Total Emissions,...,Freshwater Withdrawals per 100g Protein (liters per 100g Protein),Freshwater Withdrawals per kilogram (liters per kilogram),Greenhouse Gas Emissions per 1000kcal (kgCO2eq per 1000kcal),Greenhouse Gas Emissions per 100g Protein (kgCO2eq per 100g Protein),Land Use per 1000kcal (m2 per 1000kcal),Land Use per Kilogram (m2 per kilogram),Land Use per 100g Protein (m2 per 100g Protein),Scarcity weighted Water Use per Kilogram (liters per kilogram),Scarcity weighted Water Use per 100g protein (liters per 100g protein),Scarcity weighted Water use per 1000kcal (liters per 1000 kilocalories)
0,Wheat & Rye (Bread),True,0.1,0.0,0.8,0.2,0.1,0.1,0.1,1.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Maize (Meal),True,0.3,0.0,0.5,0.1,0.1,0.1,0.0,1.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Barley (Beer),True,0.0,0.0,0.2,0.1,0.0,0.5,0.3,1.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Oatmeal,True,0.0,0.0,1.4,0.0,0.1,0.1,0.0,1.6,...,371.076923,482.4,0.945482,1.907692,2.897446,7.6,5.846154,18786.2,14450.92308,7162.104461
4,Rice,True,0.0,0.0,3.6,0.1,0.1,0.1,0.1,4.0,...,3166.760563,2248.4,1.207271,6.267606,0.759631,2.8,3.943662,49576.3,69825.77465,13449.891480


In [18]:
data.columns

Index(['Food product', 'Plant based', 'Land Use Change', 'Animal Feed', 'Farm',
       'Processing', 'Transport', 'Packaging', 'Retail', 'Total Emissions',
       'Eutrophying Emissions per 1000kcal (gPO4eq per 1000kcal)',
       'Eutrophying Emissions per Kilogram (gPO4eq per kilogram)',
       'Eutrophying Emissions per 100g protein (gPO4eq per 100 grams protein)',
       'Freshwater Withdrawals per 1000kcal (liters per 1000kcal)',
       'Freshwater Withdrawals per 100g Protein (liters per 100g Protein)',
       'Freshwater Withdrawals per kilogram (liters per kilogram)',
       'Greenhouse Gas Emissions per 1000kcal (kgCO2eq per 1000kcal)',
       'Greenhouse Gas Emissions per 100g Protein (kgCO2eq per 100g Protein)',
       'Land Use per 1000kcal (m2 per 1000kcal)',
       'Land Use per Kilogram (m2 per kilogram)',
       'Land Use per 100g Protein (m2 per 100g Protein)',
       'Scarcity weighted Water Use per Kilogram (liters per kilogram)',
       'Scarcity weighted Water Use p

In [19]:
# Imputation of the missing values via arithmetic mean
data = data.fillna(data.mean(numeric_only=True))

# **Example diet comparison**

Let us now compare two samples of an average omnivor diet and an average plant based diet, in order to see the difference in the emissions.

Biggest UK health website suggests 2000kcal for an average woman: [NHS UK](https://www.nhs.uk/live-well/healthy-weight/managing-your-weight/understanding-calories/#:~:text=As%20a%20guide%2C%20an%20average,physical%20activity%2C%20among%20other%20factors).

The division among macronutrients is taken as follows from [healthline](https://www.healthline.com/nutrition/best-macronutrient-ratio):

*   45–65% of the calories from carbs (225g-325g in our case)
*   20–35% of the calories from fats (45g-78g in our case)
*   10–35% of the calories from proteins (50g-175g in our case)

*Remark*: the following diets are descripted by the amount of food category consumed in a day, not considering particular recipes or meals, nore guidelines about correct division in ingredients categories. The nutritional facts of the different foods are taken from various famous health websites.

**Example of a day of omnivor eating**:

400 grams white bread ------------------------------- 1060 kcal 12.8 f 196 c 36 p\
200 grams pig meat (raw) --------------------------- 314 kcal 16 f 0 c 42.6 p\
300 grams bananas ----------------------------------- 172 kcal 0.9 f 46.2 c 3.6 p\
150 grams cheese (for example mozzarella) ---- 450 kcal 33.5 f 3.3 c 33.3 p

Tot: 1996 kcal 63.2 f 245.5 c 115.5 p

In [21]:
# List of tuples (food, grams) of the diet
diet = [("Wheat & Rye (Bread)", 400), 
        ("Pig Meat", 200),
        ("Bananas", 300),
        ("Cheese", 150)]

In [22]:
# Function to sum the emission of every food in the diet, with the relative quantity
def calculate_diet_emissions(diet, data):
  tot_emissions = 0
  for item in diet:
    food_emissions = data[data["Food product"] == item[0]]["Total Emissions"]
    item_emissions = food_emissions * item[1] / 1000
    tot_emissions += item_emissions.values[0]
  return tot_emissions

In [23]:
emissions_omnivor = calculate_diet_emissions(diet, data)
print("The example omnivor diet produces " + str(emissions_omnivor) + "kg of CO2 per day on average.")

The example omnivor diet produces 5.42kg of CO2 per day on average.


**Example of a day of plant based eating**:

400 grams white bread ------------------------------- 1060 kcal 12.8 f 196 c 36 p\
300 grams bananas ------------------------------------- 172 kcal 0.9 f 46.2 c 3.6 p\
300 grams tofu --------------------------------------- 234 kcal 14.4 f 2.1 c 24 p\
200 grams peas --------------------------------------- 162 kcal 0.8 f 28.9 c 10.8 p\
50 grams nuts ---------------------------------------- 292 kcal 28.8 f 10 c 10 p\
150 grams soymilk ------------------------------------ 81 kcal 2.7 f 9 c 5 p 

Tot: 2001 kcal 60.4 f 292.2 c 89.4 p

In [24]:
# List of tuples (food, grams) of the diet
diet = [("Wheat & Rye (Bread)", 400), 
        ("Bananas", 300),
        ("Tofu", 300),
        ("Peas", 200),
        ("Nuts", 50),
        ("Soymilk", 150)]

In [25]:
emissions_plant = calculate_diet_emissions(diet, data)
print("The example plant based diet produces " + str(emissions_plant) + "kg of CO2 per day on average.")

The example plant based diet produces 2.02kg of CO2 per day on average.


In [26]:
ratio = emissions_omnivor/emissions_plant*100
round(ratio, 0)

268.0

This example shows that, in this particular case, the omnivor daily diet produces $268\%$ of the daily emissions of the plant based diet. Of course, these diets do not generalize all the different cases that can happen, given the remark written above and given every individual different needs (e.g. athletes).